# Spotify Season Playlist Maker

* Go to this website and get Oath toke - https://developer.spotify.com/console/get-current-user-saved-tracks/?market=&limit=&offset=
* Allow permissions to user-library-read, user-read-email, user-follow-read, playlist-modify-private, playlist-modify-public
* Assign the token to the "token" variable
* Select Cell -> Run Cells

# Enter User Name

In [ ]:
user_name = "Sample"
begin_month = 9
end_month = 12

# Insert Token

In [ ]:
token = ""


# Select Cell -> Run Cells to Generate your Playlist

In [ ]:
from spotify_api import get_spotify, concat_df, date_parse_df
import json
import pandas as pd
pd.set_option('display.max_columns', 25)
import requests

In [ ]:
help(get_spotify)

In [ ]:
user_likes = get_spotify(token,kind="user_tracks",user=True)


In [ ]:
# Concat all the list of responses

df_likes = concat_df(user_likes)

In [ ]:
df_likes

In [ ]:
# parse out the items column
df_items = pd.json_normalize(df_likes["items"])

df_items

In [ ]:
# Get 1 artist name from each record - Improvement can be made to get other artist, but this will suffice
df_items["artist_name"] = df_items["track.album.artists"].apply(lambda x: x[0]["name"])
df_items["artist_id"] = df_items["track.album.artists"].apply(lambda x: x[0]["id"])

In [ ]:
df_items

In [ ]:
# Reorganize Columns

df_columns = ['added_at','track.album.release_date','track.album.release_date_precision',
       'track.album.name','track.album.total_tracks',
       'track.album.type','artist_name','track.name', 'track.popularity',
       'track.duration_ms','track.disc_number',
       'track.explicit', 'track.external_ids.isrc',
       'track.external_urls.spotify', 'track.href',
       'track.is_local','track.preview_url',
       'track.track_number', 'track.type',
        'track.album.album_type','track.album.href', 'track.album.id', 
        'track.id','track.uri','track.album.images','track.album.available_markets','track.available_markets', 
        'track.album.external_urls.spotify' ,'track.album.uri','track.album.artists','track.artists',"artist_id"]

In [ ]:
df_items[df_columns]

# Get Artist Info

In [ ]:
unique_artists = df_items["artist_id"].unique().tolist()

In [ ]:
help(get_spotify)

In [ ]:
#Spotify_track_uri_limit - amount of uris allowed to be in endpoint
#Cycle - is how many times around the loop

#This should be added to the library
import math
import numpy as np

spotify_track_uri_limit = 50
cycle = math.ceil((len(unique_artists)/spotify_track_uri_limit))

#Split list_track_uri into chunks by cycle amount
track_uri_chunks = np.array_split(np.array(unique_artists),cycle)

artist_info_list = []

for i in range(cycle):
    char = '%2C'
    string_track_uri_chunks = char.join(track_uri_chunks[i])
    artist_info = get_spotify(token,kind="artists",elements=string_track_uri_chunks)
    artist_info_list.append(artist_info)
    

In [ ]:
df_artist_info = pd.json_normalize(concat_df(artist_info_list)["artists"])

# Get Track Audio Features

In [ ]:
unique_tracks = df_items["track.id"].unique().tolist()

In [ ]:
df_items.columns

In [ ]:
#Spotify_track_uri_limit - amount of uris allowed to be in endpoint
#Cycle - is how many times around the loop

#This should be added to the library
import math
import numpy as np

spotify_track_uri_limit = 100
cycle = math.ceil((len(unique_tracks)/spotify_track_uri_limit))

#Split list_track_uri into chunks by cycle amount
track_uri_chunks = np.array_split(np.array(unique_tracks),cycle)

audio_features_list = []

for i in range(cycle):
    char = '%2C'
    string_track_uri_chunks = char.join(track_uri_chunks[i])
    artist_info = get_spotify(token,kind="audio-features",elements=string_track_uri_chunks)
    audio_features_list.append(artist_info)
    

In [ ]:
df_audio_features = pd.json_normalize(concat_df(audio_features_list)["audio_features"])

In [ ]:
df_audio_features

# Merge Artist Info and Track Audio Feature DFs

In [ ]:
sim_cols_artist = set(df_artist_info.columns).intersection(df_items.columns)
sim_cols_artist

In [ ]:
df_user_likes_complete = (
                            df_items
    
                          .merge(df_artist_info[["id","popularity","genres","followers.total"]],
                                 
                                 left_on="artist_id",right_on="id")
    
                          .merge(df_audio_features[["id","danceability","energy","key","loudness","mode"
                                                   ,"speechiness","acousticness","instrumentalness"
                                                   ,"liveness","valence","tempo","time_signature"]]
                                 
                                 ,left_on="track.id",right_on="id")
    
                         ).sort_values("date",ascending=False)

In [ ]:
df_user_likes_complete

In [ ]:
df_user_likes_complete.to_csv(f"{user_name}_likes.csv",index=False)

# Specify Date

In [ ]:
# Create sep cols for year, month, and day
df_user_likes_complete_date_parse = date_parse_df(df_user_likes_complete,["track.album.release_date"])

In [ ]:
# Query month_range

df_user_likes_complete_date_range = (df_user_likes_complete_date_parse.query('`track.album.release_date.month` == @begin_month & `track.album.release_date.month` <= @end_month')
.sort_values("track.album.release_date.year",ascending=False)).reset_index(drop=True)

In [ ]:
df_user_likes_complete_date_range.to_csv(f"{user_name}_likes_dr_{begin_month}_{end_month}.csv",index=False)

# Add Playlist to User Library

In [ ]:
header = {"Authorization":'Bearer '+ token}

user_profile = requests.get("https://api.spotify.com/v1/me",headers=header)

user_id = user_profile.json()["id"]

In [ ]:
user_id

In [ ]:
p_name = f"Likes_form_{begin_month}_{end_month}"
p_description = "Made for the season"

rb = {
  "name": p_name,
  "description": p_description,
  "public": False
}

playlist_endpoint = f"https://api.spotify.com/v1/users/{user_id}/playlists"


playlist_endpoint

In [ ]:
response = requests.post(playlist_endpoint,headers=header,json=rb)

In [ ]:
playlist_id = response.json()["id"]

In [ ]:
added_tracks_to_playlist = df_user_likes_complete_date_range["track.uri"].unique().tolist()

spotify_track_uri_limit = 100
cycle = math.ceil((len(added_tracks_to_playlist)/spotify_track_uri_limit))

#Split list_track_uri into chunks by cycle amount
track_uri_chunks = np.array_split(np.array(added_tracks_to_playlist),cycle)

audio_features_list = []

for i in range(cycle):
    char = ','
    string_track_uri_chunks = char.join(track_uri_chunks[i])
    add_playlist_tracks_endpoint = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?uris={string_track_uri_chunks}"
    response = requests.post(add_playlist_tracks_endpoint,headers=header)

# EDA

In [ ]:
(df_daterange_likes_complete.select_dtypes(include=[int,float]).corr()
 .style.applymap(lambda x: 'background-color : green' if x>0.30 else ''))

# OAuth

In [ ]:
"""

with open("creds.json") as f:
    creds = json.load(f)
    
    CLIENT_ID = creds["Client_ID"]
    CLIENT_SECRET = creds["Client_Secret"]
    
scope = " ".join(["user-library-read","user-read-email","user-read-private"
         ,"user-follow-read","playlist-modify-private", "playlist-modify-public"])

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL,
    {'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'scope':scope})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
token = auth_response_data['access_token']
"""    